In [1]:
using Laplacians

In [2]:
include("/Users/kimonfountoulakis/Desktop/Box Sync/University/Berkeley/postdoc/myPapers/Laplacians.jl/src/flowUtils.jl")

cutCapacity

In [271]:
include("/Users/kimonfountoulakis/Desktop/Box Sync/University/Berkeley/postdoc/myPapers/Laplacians.jl/src/min_cost_flow.jl")
#include("/Users/spielman/Laplacians/src/primalDualIPM.jl")
#include("/Users/spielman/Laplacians/src/max_flow_IPM.jl")

makeAdj (generic function with 1 method)

We introduce a data type for minimum cost flow problems.
It has an edge list, a vector of capacities of edges, a vector of costs, and a vector of demands.
We test this with the graph on 4 vertices with source 1 and destination 4.
We try to flow 1 unit of flow.  
The capacities on all edges are 0.7.
The costs on route 1-2-4 are 1, and on route 1-3-4 are 2.
So, it should flow most on route 1-2-4, as it does.

In [272]:
edges = [1 2; 2 4; 1 3; 3 4]
caps = ones(4)*0.7
costs = [1.0; 1; 2 ; 2]
dems = [1.0;0;0;-1]

mcfp = MCFproblem(edges, caps, costs, dems)

MCFproblem{Float64,Int64}([1 2; 2 4; 1 3; 3 4],[0.7,0.7,0.7,0.7],[1.0,1.0,2.0,2.0],[1.0,0.0,0.0,-1.0])

In [273]:
sol = min_cost_flow(mcfp)
flow = sol[1]

Iteration 1, ||r_p||/||b||=0.039052, ||r_d||/||c||=1.111421, rel. gap=0.313158, alpha=1.000000
Iteration 2, ||r_p||/||b||=0.034663, ||r_d||/||c||=0.986494, rel. gap=0.277381, alpha=0.980100
Iteration 3, ||r_p||/||b||=0.030987, ||r_d||/||c||=0.881889, rel. gap=0.249360, alpha=0.980100
Iteration 4, ||r_p||/||b||=0.027859, ||r_d||/||c||=0.792860, rel. gap=0.226818, alpha=0.980100
Iteration 5, ||r_p||/||b||=0.025176, ||r_d||/||c||=0.716489, rel. gap=0.208323, alpha=0.980100
Iteration 6, ||r_p||/||b||=0.022872, ||r_d||/||c||=0.650943, rel. gap=0.192957, alpha=0.980100
Iteration 7, ||r_p||/||b||=0.020906, ||r_d||/||c||=0.594983, rel. gap=0.180116, alpha=0.980100
Iteration 8, ||r_p||/||b||=0.019242, ||r_d||/||c||=0.547617, rel. gap=0.169372, alpha=0.980100
Iteration 9, ||r_p||/||b||=0.017845, ||r_d||/||c||=0.507877, rel. gap=0.160396, alpha=0.980100
Iteration 10, ||r_p||/||b||=0.016681, ||r_d||/||c||=0.474726, rel. gap=0.152906, alpha=0.980100
Iteration 11, ||r_p||/||b||=0.015709, ||r_d||/||c

4-element Array{Float64,1}:
 0.7
 0.7
 0.3
 0.3

The following cell computes the costs and check that the demands are satisfied.

In [274]:
reportMCFresults(mcfp,flow)

Cost: 2.599999850083979
Min flow: 0.2999999832706952
Min slack: 4.149940124875684e-8
Error on demands: 1.1645741221366634e-7


Now, let's try computing this using JuMP, an optimization package for Julia.

In [275]:
using JuMP
using Clp

In [276]:
function MCFjump(mcfp::MCFproblem)
    edge_list = mcfp.edge_list
    m = size(edge_list,1)
    n = maximum(edge_list)
    B = sparse(collect(1:m), edge_list[:,1], 1.0, m, n) -
      sparse(collect(1:m), edge_list[:,2], 1.0, m, n)
 

    mod = Model(solver=ClpSolver(PrimalTolerance=1.0e-1,DualTolerance=1.0e-1))
    @variable(mod, x[1:m] >= 0)
    @constraint(mod, x .<= mcfp.capacities)
    @constraint(mod, B'*x .== mcfp.demands)
    @objective(mod, Min, (mcfp.costs'*x)[1])
    @time status = solve(mod)

    f = getvalue(x)
    return f
    
end

MCFjump (generic function with 1 method)

In [277]:
f = MCFjump(mcfp)
reportMCFresults(mcfp,f)

  0.000386 seconds (110 allocations: 6.969 KB)
Cost: 2.6
Min flow: 0.30000000000000004
Min slack: 0.0
Error on demands: 0.0


We would like to compare the performance of our code to standard codes on some benchmark examples.  These benchmark examples usually come in the Dimacs format.  
In this notebook, we use the example "goto_8_08a.min"
from <a href="http://lime.cs.elte.hu/~kpeter/data/mcf/goto/">http://lime.cs.elte.hu/~kpeter/data/mcf/goto/</a>.

In [278]:
mcfp = readDimacsMCF("/Users/kimonfountoulakis/Desktop/temp/goto_8_08a.min")

MCFproblem{Float64,Int64}([1 2; 1 3; … ; 180 216; 216 252],[8642.0,741.0,453.0,62.0,341.0,1029.0,2640.0,1762.0,155.0,31.0  …  55614.0,55614.0,55614.0,55614.0,55614.0,55614.0,55614.0,55614.0,55614.0,55614.0],[28.0,714.0,451.0,822.0,0.0,7.0,518.0,29.0,298.0,973.0  …  142.0,142.0,142.0,142.0,142.0,142.0,142.0,142.0,142.0,142.0],[55614.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,-55614.0,0.0,0.0,0.0,0.0])

In [280]:
#@time sol = min_cost_flow(mcfp, lapSolver = (h->augTreeLap(h, verbose=false)), tol=1e-9)
@time sol = min_cost_flow(mcfp, lapSolver = cholLap, tol = 1.0e-12)
flow = sol[1]
reportMCFresults(mcfp,flow)

Iteration 1, ||r_p||/||b||=1.049937, ||r_d||/||c||=57.144463, rel. gap=0.136245, alpha=1.000000
Iteration 2, ||r_p||/||b||=1.049605, ||r_d||/||c||=57.126426, rel. gap=0.136243, alpha=0.000331
Iteration 3, ||r_p||/||b||=1.048901, ||r_d||/||c||=57.088082, rel. gap=0.136238, alpha=0.000704
Iteration 4, ||r_p||/||b||=1.046585, ||r_d||/||c||=56.962012, rel. gap=0.136225, alpha=0.002323
Iteration 5, ||r_p||/||b||=1.039698, ||r_d||/||c||=56.587220, rel. gap=0.136196, alpha=0.006994
Iteration 6, ||r_p||/||b||=1.017787, ||r_d||/||c||=55.394673, rel. gap=0.136106, alpha=0.023022
Iteration 7, ||r_p||/||b||=0.945914, ||r_d||/||c||=51.482861, rel. gap=0.135308, alpha=0.081554
Iteration 8, ||r_p||/||b||=0.703305, ||r_d||/||c||=38.278500, rel. gap=0.127741, alpha=0.324087
Iteration 9, ||r_p||/||b||=0.299752, ||r_d||/||c||=16.314484, rel. gap=0.085410, alpha=0.980100
Iteration 10, ||r_p||/||b||=0.152051, ||r_d||/||c||=8.275592, rel. gap=0.056265, alpha=0.980100
Iteration 11, ||r_p||/||b||=0.116712, ||

LoadError: Base.LinAlg.PosDefException(254)

In [231]:
@time sol = min_cost_flow(mcfp, lapSolver = (h->augTreeLap(h, tol=1e-12)), tol=1e-11)
flow = sol[1]
reportMCFresults(mcfp,flow)

Iteration 1, ||r_p||/||b||=1.049937, ||r_d||/||c||=57.144463, rel. gap=0.136245, alpha=1.000000
Iteration 2, ||r_p||/||b||=1.049262, ||r_d||/||c||=57.107740, rel. gap=0.136204, alpha=0.000330
Iteration 3, ||r_p||/||b||=1.047827, ||r_d||/||c||=57.029612, rel. gap=0.136118, alpha=0.000704
Iteration 4, ||r_p||/||b||=1.043102, ||r_d||/||c||=56.772448, rel. gap=0.135837, alpha=0.002324
Iteration 5, ||r_p||/||b||=1.028991, ||r_d||/||c||=56.004454, rel. gap=0.135013, alpha=0.007006
Iteration 6, ||r_p||/||b||=0.983571, ||r_d||/||c||=53.532379, rel. gap=0.132325, alpha=0.023166
Iteration 7, ||r_p||/||b||=0.829276, ||r_d||/||c||=45.134631, rel. gap=0.121805, alpha=0.084741
Iteration 8, ||r_p||/||b||=0.487998, ||r_d||/||c||=26.560056, rel. gap=0.087455, alpha=0.240112
Iteration 9, ||r_p||/||b||=0.222508, ||r_d||/||c||=12.110338, rel. gap=0.047165, alpha=0.358938
Iteration 10, ||r_p||/||b||=0.080002, ||r_d||/||c||=4.354243, rel. gap=0.019614, alpha=0.450106
Iteration 11, ||r_p||/||b||=0.011454, ||

As a sanity check, we compare the results from MCFjump.

In [148]:
f = MCFjump(mcfp)
mcfp.costs'*f

  0.013704 seconds (102 allocations: 671.797 KB)


1-element Array{Float64,1}:
 5.60871e8

This is very similar.  But, JuMP computed a flow of a slightly lower cost.
This should not be surprising, because we haven't figured out exactly what tol we want.
It might help to know that all the numbers in the Dimacs problems are integers.

In [149]:
mcfp.costs'*f - mcfp.costs'*flow

1-element Array{Float64,1}:
 -0.867158

For comparison, we should also try the code from "Lemon", available at

<a href="http://lemon.cs.elte.hu/trac/lemon">http://lemon.cs.elte.hu/trac/lemon</a>

It was easy to install on my Mac (I just needed cmake, which I got by `brew install cmake`)

Here is a transcript of running it:

```
Dans17:tools spielman$ ./dimacs-solver ~/tmp/goto_8_08a.min 
Problem type: min
Num of nodes: 256
Num of arcs:  2048

Sum of supply values: 0
GEQ supply contraints are used for NetworkSimplex

Read the file: u: 0s, s: 0s, cu: 0s, cs: 0s, real: 0.00467205s
Setup NetworkSimplex class: u: 0s, s: 0s, cu: 0s, cs: 0s, real: 7.70092e-05s
Run NetworkSimplex: u: 0s, s: 0s, cu: 0s, cs: 0s, real: 0.00143886s

Feasible flow: found
Min flow cost: 560870539
```

## More Tests

Let's just do more tests on the GOTO problems, just to get a feel for the speeds involved.  Keep in mind that different problems will have very different run times, and that our code can be sped up a lot.

The Laplacian systems generated by the IPMs can have edge weights varying by 12 orders of magnitude!  This causes some of the solvers to run into numerical issues.  This is interesting to me, as it points out a place where our solvers need more development.

In [150]:
mcfp = readDimacsMCF("/Users/spielman/tmp/goto_8_13a.min")

LoadError: SystemError: opening file /Users/spielman/tmp/goto_8_13a.min: No such file or directory

In [151]:
flow = MCFjump(mcfp)
reportMCFresults(mcfp, flow)

  0.012271 seconds (102 allocations: 671.797 KB)
Cost: 5.60870539e8
Min flow: -0.0
Min slack: 0.0
Error on demands: 0.0


In [152]:
@time sol = min_cost_flow(mcfp, lapSolver = cholLap, tol = 1e-5)
flow = sol[1]
reportMCFresults(mcfp,flow)

Iteration 1, ||r_p||/||b||=1.049937, ||r_d||/||c||=57.144463, rel. gap=0.272490, alpha=1.000000
Iteration 2, ||r_p||/||b||=1.049599, ||r_d||/||c||=57.126093, rel. gap=0.272484, alpha=0.000165
Iteration 3, ||r_p||/||b||=1.048881, ||r_d||/||c||=57.087018, rel. gap=0.272473, alpha=0.000352
Iteration 4, ||r_p||/||b||=1.046517, ||r_d||/||c||=56.958343, rel. gap=0.272436, alpha=0.001161
Iteration 5, ||r_p||/||b||=1.039452, ||r_d||/||c||=56.573799, rel. gap=0.272338, alpha=0.003497
Iteration 6, ||r_p||/||b||=1.016657, ||r_d||/||c||=55.333139, rel. gap=0.271961, alpha=0.011509
Iteration 7, ||r_p||/||b||=0.939708, ||r_d||/||c||=51.145108, rel. gap=0.269187, alpha=0.040778
Iteration 8, ||r_p||/||b||=0.666896, ||r_d||/||c||=36.296887, rel. gap=0.245563, alpha=0.163022
Iteration 9, ||r_p||/||b||=0.308550, ||r_d||/||c||=16.793330, rel. gap=0.157778, alpha=0.338202
Iteration 10, ||r_p||/||b||=0.128807, ||r_d||/||c||=7.010497, rel. gap=0.084705, alpha=0.393424
Iteration 11, ||r_p||/||b||=0.019362, ||

In [153]:
@time sol = min_cost_flow(mcfp, lapSolver = (h->augTreeLap(h, tol=1e-8, verbose=true, maxits=1000)), tol=1e-6)
flow = sol[1]
reportMCFresults(mcfp, flow)

Iteration 1, ||r_p||/||b||=1.049937, ||r_d||/||c||=57.144463, rel. gap=0.272490, alpha=1.000000
Solver build time: 0.008 seconds.
PCG BLAS stopped after: 0.001 seconds and 32 iterations with relative error 7.85291442530764e-9.
PCG BLAS stopped after: 0.001 seconds and 32 iterations with relative error 6.17655696172033e-9.
Iteration 2, ||r_p||/||b||=1.049599, ||r_d||/||c||=57.126093, rel. gap=0.272484, alpha=0.000165
Solver build time: 0.002 seconds.
PCG BLAS stopped after: 0.001 seconds and 34 iterations with relative error 8.410680463621335e-9.
PCG BLAS stopped after: 0.001 seconds and 34 iterations with relative error 7.3928323797411796e-9.
Iteration 3, ||r_p||/||b||=1.048881, ||r_d||/||c||=57.087018, rel. gap=0.272473, alpha=0.000352
Solver build time: 0.002 seconds.
PCG BLAS stopped after: 0.006 seconds and 32 iterations with relative error 5.21412667573984e-9.
PCG BLAS stopped after: 0.001 seconds and 32 iterations with relative error 4.943237553921054e-9.
Iteration 4, ||r_p||/||b

Here is what Lemon did:

```
Dans17:tools spielman$ ./dimacs-solver ~/tmp/goto_8_13a.min 
Problem type: min
Num of nodes: 8192
Num of arcs:  65536

Sum of supply values: 0
GEQ supply contraints are used for NetworkSimplex

Read the file: u: 0.08s, s: 0s, cu: 0s, cs: 0s, real: 0.0842741s
Setup NetworkSimplex class: u: 0s, s: 0s, cu: 0s, cs: 0s, real: 0.00165105s
Run NetworkSimplex: u: 0.43s, s: 0s, cu: 0s, cs: 0s, real: 0.439616s

Feasible flow: found
Min flow cost: 18217956686
```

It was absurdly faster.  But, I won't worry about that yet.


Here is how Goldberg's CS2 code does, 
which I obtained from

<a href="https://github.com/iveney/cs2">https://github.com/iveney/cs2</a>

```
Dans17:cs2 spielman$ time ./cs2 < ~/tmp/goto_8_13a.min > out.txt
warning: this program uses gets(), which is unsafe.

real	0m2.425s
user	0m2.344s
sys	0m0.027s

Dans17:cs2 spielman$ head out.txt 
c CS 4.6
c Commercial use requires a licence
c contact igsys@eclipse.net
c
c nodes:            8192     arcs:            65536
c scale-factor:       12     cut-off-factor:   79.1
c
c time:             2.25     cost:      18217956686
c refines:             4     discharges:    3409417
c pushes:        3621130     relabels:      1381573
```

## goto_8_14b.min

In [267]:
mcfp = readDimacsMCF("/Users/kimonfountoulakis/Desktop/temp/goto_8_14b.min")

MCFproblem{Float64,Int64}([1 2; 1 3; … ; 15120 15750; 15750 16380],[2481.0,951.0,346.0,61.0,8368.0,9891.0,6933.0,783.0,155.0,37.0  …  202084.0,202084.0,202084.0,202084.0,202084.0,202084.0,202084.0,202084.0,202084.0,202084.0],[807.0,189.0,40.0,285.0,4.0,5.0,323.0,675.0,464.0,19.0  …  38.0,38.0,38.0,38.0,38.0,38.0,38.0,38.0,38.0,38.0],[202084.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,-202084.0,0.0,0.0,0.0,0.0])

Some of these codes run into numerical issues at very low tolerances.  So, I will be more generous for this run.

In [270]:
@time sol = min_cost_flow(mcfp, lapSolver = (h->augTreeLap(h,tol=1.0e-12)), tol=1e-12)
flow = sol[1]
reportMCFresults(mcfp, flow)

Iteration 1, ||r_p||/||b||=2.485507, ||r_d||/||c||=235.856325, rel. gap=0.025382, alpha=1.000000
Iteration 2, ||r_p||/||b||=2.485088, ||r_d||/||c||=235.816619, rel. gap=0.025383, alpha=0.000086
Iteration 3, ||r_p||/||b||=2.484212, ||r_d||/||c||=235.733448, rel. gap=0.025384, alpha=0.000180
Iteration 4, ||r_p||/||b||=2.481308, ||r_d||/||c||=235.457870, rel. gap=0.025386, alpha=0.000598
Iteration 5, ||r_p||/||b||=2.472787, ||r_d||/||c||=234.649314, rel. gap=0.025382, alpha=0.001758
Iteration 6, ||r_p||/||b||=2.446579, ||r_d||/||c||=232.162421, rel. gap=0.025332, alpha=0.005444
Iteration 7, ||r_p||/||b||=2.365324, ||r_d||/||c||=224.451891, rel. gap=0.025054, alpha=0.017204
Iteration 8, ||r_p||/||b||=2.108854, ||r_d||/||c||=200.114732, rel. gap=0.023826, alpha=0.056972
Iteration 9, ||r_p||/||b||=1.390452, ||r_d||/||c||=131.943639, rel. gap=0.019190, alpha=0.183349
Iteration 10, ||r_p||/||b||=1.073522, ||r_d||/||c||=101.869325, rel. gap=0.016680, alpha=0.122330
Iteration 11, ||r_p||/||b||=0

LoadError: Base.LinAlg.PosDefException(16382)

In [269]:
@time sol = min_cost_flow(mcfp, lapSolver = cholLap , tol=1.0e-11)
flow = sol[1]
reportMCFresults(mcfp, flow)

Iteration 1, ||r_p||/||b||=2.485507, ||r_d||/||c||=235.856325, rel. gap=0.025382, alpha=1.000000
Iteration 2, ||r_p||/||b||=2.485088, ||r_d||/||c||=235.816619, rel. gap=0.025383, alpha=0.000086
Iteration 3, ||r_p||/||b||=2.484212, ||r_d||/||c||=235.733448, rel. gap=0.025384, alpha=0.000180
Iteration 4, ||r_p||/||b||=2.481308, ||r_d||/||c||=235.457870, rel. gap=0.025386, alpha=0.000598
Iteration 5, ||r_p||/||b||=2.472787, ||r_d||/||c||=234.649314, rel. gap=0.025382, alpha=0.001758
Iteration 6, ||r_p||/||b||=2.446579, ||r_d||/||c||=232.162421, rel. gap=0.025332, alpha=0.005444
Iteration 7, ||r_p||/||b||=2.365324, ||r_d||/||c||=224.451891, rel. gap=0.025054, alpha=0.017204
Iteration 8, ||r_p||/||b||=2.108854, ||r_d||/||c||=200.114732, rel. gap=0.023826, alpha=0.056972
Iteration 9, ||r_p||/||b||=1.390452, ||r_d||/||c||=131.943639, rel. gap=0.019190, alpha=0.183349
Iteration 10, ||r_p||/||b||=1.073522, ||r_d||/||c||=101.869325, rel. gap=0.016680, alpha=0.122330
Iteration 11, ||r_p||/||b||=0

LoadError: Base.LinAlg.PosDefException(16382)

If I try tol = 1e-4, that solver dies due to numerical issues.  

And, here is Lemon:

```
Dans17:tools spielman$ ./dimacs-solver ~/tmp/goto_8_14b.min 
Problem type: min
Num of nodes: 16384
Num of arcs:  131072

Sum of supply values: 0
GEQ supply contraints are used for NetworkSimplex

Read the file: u: 0.18s, s: 0s, cu: 0s, cs: 0s, real: 0.187166s
Setup NetworkSimplex class: u: 0s, s: 0s, cu: 0s, cs: 0s, real: 0.00391507s
Run NetworkSimplex: u: 1.6s, s: 0.01s, cu: 0s, cs: 0s, real: 1.61133s

Feasible flow: found
Min flow cost: 41834525746
```


And, here is CS2:

```
Dans17:cs2 spielman$ time ./cs2 < ~/tmp/goto_8_14b.min > out.txt
warning: this program uses gets(), which is unsafe.

real	0m22.478s
user	0m21.870s
sys	0m0.096s
Dans17:cs2 spielman$ head out.txt 
c CS 4.6
c Commercial use requires a licence
c contact igsys@eclipse.net
c
c nodes:           16384     arcs:           131072
c scale-factor:       12     cut-off-factor:  107.3
c
c time:            21.70     cost:      41834525746
c refines:             5     discharges:    9131788
c pushes:        9571379     relabels:      3496218
```

## goto_8_16a
Lemon (which is implementing a network simplex code) starts to scale badly when given this problem, as does CS2.
Our scales as we would expect.
This problem is 4 times bigger than `goto_8_14b`.

I stopped CS2 after 7 minutes, because it was driving me nuts.
Here is how Lemon did:

```
Dans17:tools spielman$ ./dimacs-solver ~/tmp/goto_8_16a.min 
Problem type: min
Num of nodes: 65536
Num of arcs:  524288

Sum of supply values: 0
GEQ supply contraints are used for NetworkSimplex

Read the file: u: 0.68s, s: 0.01s, cu: 0s, cs: 0s, real: 0.711415s
Setup NetworkSimplex class: u: 0.01s, s: 0.01s, cu: 0s, cs: 0s, real: 0.015645s
Run NetworkSimplex: u: 80.66s, s: 0.26s, cu: 0s, cs: 0s, real: 81.4035s
```

In [4]:
mcfp = readDimacsMCF("/Users/spielman/tmp/goto_8_16a.min")

MCFproblem{Float64,Int64}([1 2; 1 3; … ; 62244 63882; 63882 65520],[8642.0,741.0,453.0,62.0,341.0,1029.0,2640.0,1762.0,155.0,31.0  …  331827.0,331827.0,331827.0,331827.0,331827.0,331827.0,331827.0,331827.0,331827.0,331827.0],[28.0,714.0,451.0,822.0,0.0,7.0,518.0,29.0,298.0,973.0  …  25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0],[331827.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])

In [36]:
@time sol = min_cost_flow(mcfp, lapSolver = cholLap, tol=1)
flow = sol[1]
reportMCFresults(mcfp, flow)

Iteration 1, ||r_p||=1370095.345474, ||r_d||=116864630.148304, mu=20862256364.106258
Iteration 2, ||r_p||=1370054.044492, ||r_d||=116861107.310279, mu=20862716994.257442
Iteration 3, ||r_p||=1369966.873185, ||r_d||=116853671.884245, mu=20863680411.510319
Iteration 4, ||r_p||=1369679.984777, ||r_d||=116829201.245920, mu=20866833498.161743
Iteration 5, ||r_p||=1368835.380943, ||r_d||=116757159.314662, mu=20875887167.123146
Iteration 6, ||r_p||=1366196.332895, ||r_d||=116532057.189363, mu=20902067158.667389
Iteration 7, ||r_p||=1357784.805709, ||r_d||=115814581.564949, mu=20972179454.468510
Iteration 8, ||r_p||=1330063.502768, ||r_d||=113450045.530183, mu=21142627083.457748
Iteration 9, ||r_p||=1236606.194701, ||r_d||=105478444.299676, mu=21468228709.443768
Iteration 10, ||r_p||=897148.187194, ||r_d||=76523791.888600, mu=21175505322.016315
Iteration 11, ||r_p||=555327.863769, ||r_d||=47367641.693511, mu=23208513306.810078
Iteration 12, ||r_p||=294413.760916, ||r_d||=25112526.214785, mu=16

In [37]:
@time sol = min_cost_flow(mcfp, lapSolver = (h->augTreeLap(h, tol=1e-10)), tol=1)
flow = sol[1]
reportMCFresults(mcfp, flow)

Iteration 1, ||r_p||=1370095.345474, ||r_d||=116864630.148304, mu=20862256364.106258
Iteration 2, ||r_p||=1370054.044492, ||r_d||=116861107.310279, mu=20862716994.257439
Iteration 3, ||r_p||=1369966.873185, ||r_d||=116853671.884245, mu=20863680411.510315
Iteration 4, ||r_p||=1369679.984777, ||r_d||=116829201.245920, mu=20866833498.161751
Iteration 5, ||r_p||=1368835.380943, ||r_d||=116757159.314662, mu=20875887167.123150
Iteration 6, ||r_p||=1366196.332895, ||r_d||=116532057.189363, mu=20902067158.667427
Iteration 7, ||r_p||=1357784.805709, ||r_d||=115814581.564951, mu=20972179454.468803
Iteration 8, ||r_p||=1330063.502768, ||r_d||=113450045.530191, mu=21142627083.458965
Iteration 9, ||r_p||=1236606.194702, ||r_d||=105478444.299744, mu=21468228709.455700
Iteration 10, ||r_p||=897148.187197, ||r_d||=76523791.888869, mu=21175505322.071014
Iteration 11, ||r_p||=555327.863778, ||r_d||=47367641.694227, mu=23208513306.774761
Iteration 12, ||r_p||=294413.760965, ||r_d||=25112526.218994, mu=16

In [159]:
include("/Users/kimonfountoulakis/Desktop/Box Sync/University/Berkeley/postdoc/myPapers/Laplacians.jl/src/matlabSolvers.jl")

INFO: Precompiling module MATLAB.


>> >> >> Connected to Matlab!


matlabCmgLap

In [160]:
@time sol = min_cost_flow(mcfp, lapSolver = (h->matlabCmgLap(h, tol=1e-8)), tol=1)
flow = sol[1]

Iteration 1, ||r_p||/||b||=2.485507, ||r_d||/||c||=235.856325, rel. gap=0.050764, alpha=1.000000


{Undefined function or variable 'cmg_sdd'.
} 
{Undefined function or variable 'pfun'.
} 
{Error using save
Variable 'iter' not found.
} 


LoadError: MATLAB.MEngineError("failed to get the variable iter from a MATLAB session")

In [10]:
reportMCFresults(mcfp, flow)

Cost: 2.0566500432805237e11
Min flow: 1.1533763144431898e-7
Min slack: 1.1589416715196421e-7
Error on demands: 1.676119310230133


In [ ]:
a = [1,2,3,4] 

In [229]:
a .> ones(4)*0.5

4-element BitArray{1}:
 true
 true
 true
 true